In [24]:
import sys
sys.path.append('/Users/rkam/projs/algotrade')
import numpy as np
import pandas as pd
import scipy as sp
import itertools
from sklearn.neighbors import NearestNeighbors
from typing import Set, Dict, Tuple, NewType
from tqdm import tqdm
from datetime import timedelta, datetime
from src.dbmanager import DBManager

In [33]:
buy_actions = list(itertools.product(['buy'], np.arange(0.2, 0.5, 0.1), np.arange(0.01, 0.11, 0.01)))
sell_actions = list(itertools.product(['sell'], np.arange(-0.5, -0.2, 0.1), np.arange(0.01, 0.11, 0.01)))
actions = set(buy_actions + sell_actions)

In [26]:
dbob = DBManager(path='/Users/rkam/projs/algotrade/data/db/ob.db')
dbtr = DBManager(path='/Users/rkam/projs/algotrade/data/db/tr.db')

## Data

In [27]:
ob = dbob.read(
    """
    select
        *
    from
        ob
    """
)
ob.columns = ['ts', 'bprice', 'bquantity', 'aprice', 'aquantity']
ob['ts'] = pd.to_datetime(ob['ts']).dt.tz_localize(None)
ob = ob.iloc[:1000000]

tr = dbtr.read(
    """
    select
        *
    from
        tr
    """
)
tr.columns = ['ts', 'direction', 'price', 'quantity']
tr['ts'] = pd.to_datetime(tr['ts']).dt.tz_localize(None)

In [5]:
ob.head()

,ts,bprice,bquantity,aprice,aquantity
0,2024-05-22 12:11:23.338882,322.30,478,322.31,911
1,2024-05-22 12:11:23.338882,322.29,372,322.33,124
2,2024-05-22 12:11:23.338882,322.28,1,322.34,10
3,2024-05-22 12:11:23.338882,322.27,45,322.35,1392
4,2024-05-22 12:11:23.338882,322.26,45,322.36,24


In [6]:
tr.head()

,ts,direction,price,quantity
0,2024-05-22 12:11:34.797401,2,322.30,100
1,2024-05-22 12:11:35.281418,1,322.31,1
2,2024-05-22 12:11:35.281418,1,322.31,2
3,2024-05-22 12:11:35.496058,2,322.30,51
4,2024-05-22 12:11:35.496058,2,322.30,2


## Формируем множества

In [7]:
depth_ob = 5
last_trades = 5
states = set()
for ts in tqdm(ob['ts'].unique(), total=ob['ts'].nunique()):
     
    cols = ['direction', 'price', 'quantity']
    trades = tr[tr['ts'] <= ts].iloc[-last_trades:]
    trades[cols] = trades[cols].apply(lambda col: col / col.iloc[-1])
    embd_tr = trades[cols].values.flatten()
    if trades.shape[0] != last_trades:
        continue
    cols = ['bprice', 'bquantity', 'aprice', 'aquantity']
    orderbook = ob[ob['ts'] == ts].iloc[:depth_ob]
    orderbook[cols] = orderbook[cols].apply(lambda col: col / col.max())
    embd_ob = orderbook[cols].values.flatten()

    state = tuple(np.concatenate([embd_ob, embd_tr]))
    states.add(state)

100%|██████████| 20008/20008 [00:55<00:00, 362.63it/s]


In [21]:
embd_length = pd.Series([len(x) for x in states]).value_counts().index[0]
states = [x for x in states if len(x) == embd_length]

In [23]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')\
    .fit(list(states))


In [36]:
states = list(states)
for _ in range(0, len(states)):
    state = states[0]
    try:
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')\
            .fit(np.array(list(states)))
        print(state)
        break
    except ValueError:
        states.remove(state)


(1.0, 0.24833702882483372, 0.9998759612999255, 0.15267175572519084, 0.9999689855162361, 0.2682926829268293, 0.9999069709749441, 1.0, 0.9999069565487082, 0.03325942350332594, 0.9999379806499626, 0.03435114503816794, 0.9998759420649442, 0.17960088691796008, 0.9999689903249814, 0.1267175572519084, 0.9998449275811804, 1.0, 1.0, 0.6450381679389313, 0.5, 0.9999689855162361, 1.4, 0.5, 0.9999689855162361, 58.6, 1.0, 0.9999379710324722, 0.8, 0.5, 1.0, 10.0, 1.0, 1.0, 1.0)
